In [1]:
import pandas as pd

In [2]:
cd ..

/Users/johanbarreiro/Documents/GitHub/vl_optimizer


In [3]:
df = pd.read_csv('data/processed_data/industrial_sites_processed/2024-07-05T01-00-40_industrial_site2_processed.csv')

# Data Center

[Server Source](https://cc-techgroup.com/data-center-energy-consumption/#:~:text=this%20work%20efficiently.-,How%20Many%20kWh%20Does%20a%20Server%20Use%3F,to%201%2C900%20kWh%20every%20year.)

Average annual power may be around 1,800 to 1,900 kWh every year per server (divide by 365 then 24 the 4)   0.05279680365 kWh per 15 minutes

[Network Switch Source](https://www.genuinemodules.com/how-much-power-does-a-24-port-switch-use_a8539#:~:text=However%2C%20as%20a%20general%20estimate,requirements%20of%20the%20connected%20devices.)


However, as a general estimate, a non-PoE 24 port switch may consume around 20-50 watts of power, while a PoE-enabled switch may consume around 100-250 watts or more, depending on the power requirements of the connected devices. 
PoE: 25Wh - 62.5Wh per 15 minutes
Non-PoE: 5Wh - 12.5Wh per 15 minutes
    
 [HDD & SDD Source](https://dataspan.com/blog/how-much-energy-do-data-centers-use/#:~:text=Depending%20on%20disk%20size%2C%20an,around%206%20watts%20per%20disk.)   

Depending on disk size, an HDD can use anywhere from 6 to 9 watts at maximum capacity. While this is lower than in previous years, power requirements for SSDs have remained mostly constant at around 6 watts per disk.

HDD: 0.175Wh - 0.2625Wh per 15 minutes
SSD: 0.175Wh per 15 minutes


In [4]:
 df.columns

Index(['Time', '01 General_Electric_Active Energy (kWh)',
       '01 General_Electric_Active Power (kW)',
       '01 General_Electric_Power Factor (real)',
       '02 Production_Electric_Active Energy (kWh)',
       '02 Production_Electric_Active Power (kW)',
       '02 Production_Electric_Power Factor (real)',
       '03 Chiller Group_Electric_Active Energy (kWh)',
       '03 Chiller Group_Electric_Active Power (kW)',
       '03 Chiller Group_Electric_Power Factor (real)',
       '04 UTA_Electric_Active Energy (kWh)',
       '04 UTA_Electric_Active Power (kW)',
       '04 UTA_Electric_Power Factor (real)',
       '05 Compressors_Electric_Active Energy (kWh)',
       '05 Compressors_Electric_Active Power (kW)',
       '05 Compressors_Electric_Power Factor (real)',
       '06 Offices_Electric_Active Energy (kWh)',
       '06 Offices_Electric_Active Power (kW)',
       '06 Offices_Electric_Power Factor (real)',
       '07 Data Center_Electric_Active Energy (kWh)',
       '07 Data Center_

In [5]:
data_center_target = (df['07 Data Center_Electric_Active Energy (kWh)'])
data_center_target.to_csv('data/target_variables/target_data_center.csv')

## Synthesized dataset containing the following columns:

total_data_center_consumption_kwh = β1*Number_of_active_servers + β2*Number_of_active_network_switches + β3*Number_of_active_hard_drives

	•	Number of active servers
	•	Number of active network switches (PoE)
	•	Number of active network switches (Non-PoE)
	•	Number of active hard drives (HDD)
	•	Number of active solid-state drives (SSD)
	•	Total energy consumption in kWh

In [6]:
import numpy as np
import pandas as pd

# Define the number of records to generate
num_records = 100  # Set to a desired number of records

# Define the energy consumption per component per 15 minutes
energy_consumption_per_15_min = {
    'server': 0.05279680365,  # kWh per 15 minutes
    'network_switch_poe': (25, 62.5),  # Wh per 15 minutes
    'network_switch_non_poe': (5, 12.5),  # Wh per 15 minutes
    'hdd': (0.175, 0.2625),  # Wh per 15 minutes
    'ssd': 0.175  # Wh per 15 minutes
}

# Generate random numbers of each component within a realistic range
np.random.seed(42)  # For reproducibility

num_servers = np.maximum(np.random.normal(50, 10, num_records).astype(int), 0)
num_network_switches_poe = np.maximum(np.random.normal(10, 2, num_records).astype(int), 0)
num_network_switches_non_poe = np.maximum(np.random.normal(5, 1, num_records).astype(int), 0)
num_hdds = np.maximum(np.random.normal(100, 20, num_records).astype(int), 0)
num_ssds = np.maximum(np.random.normal(50, 10, num_records).astype(int), 0)

# Calculate total energy consumption for each record
total_energy_consumption_kwh = (
    num_servers * energy_consumption_per_15_min['server'] +
    num_network_switches_poe * np.random.uniform(energy_consumption_per_15_min['network_switch_poe'][0], energy_consumption_per_15_min['network_switch_poe'][1], num_records) / 1000 +
    num_network_switches_non_poe * np.random.uniform(energy_consumption_per_15_min['network_switch_non_poe'][0], energy_consumption_per_15_min['network_switch_non_poe'][1], num_records) / 1000 +
    num_hdds * np.random.uniform(energy_consumption_per_15_min['hdd'][0], energy_consumption_per_15_min['hdd'][1], num_records) / 1000 +
    num_ssds * energy_consumption_per_15_min['ssd'] / 1000
)

# Create a DataFrame with the synthesized data
synthesized_data = pd.DataFrame({
    'Number_of_active_servers': num_servers,
    'Number_of_active_network_switches_poe': num_network_switches_poe,
    'Number_of_active_network_switches_non_poe': num_network_switches_non_poe,
    'Number_of_active_hard_drives': num_hdds,
    'Number_of_active_ssds': num_ssds,
    'Total_energy_consumption_kwh': total_energy_consumption_kwh
})

# Display the synthesized data
print(synthesized_data.head())

# Save the synthesized data to a CSV file
synthesized_data.to_csv('data/synthesized_data/data_center_synthesized.csv', index=False)

   Number_of_active_servers  Number_of_active_network_switches_poe  \
0                        54                                      7   
1                        48                                      9   
2                        56                                      9   
3                        65                                      8   
4                        47                                      9   

   Number_of_active_network_switches_non_poe  Number_of_active_hard_drives  \
0                                          5                            83   
1                                          5                            88   
2                                          6                           114   
3                                          6                           112   
4                                          3                            99   

   Number_of_active_ssds  Total_energy_consumption_kwh  
0                     34                      3.27867

# Total energy Consumption Equation

The total energy consumption ( P ) can be represented as:

\[ P = \beta_1 \cdot N_{\text{servers}} + \beta_2 \cdot N_{\text{network\_switches\poe}} + \beta_3 \cdot N{\text{network\_switches\_non\poe}} + \beta_4 \cdot N{\text{hard\drives}} + \beta_5 \cdot N{\text{ssds}} \]

Where:

	•	 N_{\text{servers}} = Number of active servers
	•	 N_{\text{network\_switches\_poe}}  = Number of active network switches (PoE)
	•	 N_{\text{network\_switches\_non\_poe}}  = Number of active network switches (Non-PoE)
	•	 N_{\text{hard\_drives}}  = Number of active hard drives (HDD)
	•	 N_{\text{ssds}}  = Number of active solid-state drives (SSD)

And the coefficients are:

	•	 \beta_1 = 0.05279680365  kWh per server per 15 minutes
	•	 \beta_2  is a range between 25 Wh to 62.5 Wh per PoE switch per 15 minutes (converted to kWh)
	•	 \beta_3  is a range between 5 Wh to 12.5 Wh per non-PoE switch per 15 minutes (converted to kWh)
	•	 \beta_4  is a range between 0.175 Wh to 0.2625 Wh per HDD per 15 minutes (converted to kWh)
	•	 \beta_5 = 0.175  Wh per SSD per 15 minutes (converted to kWh)


In [7]:
from pyomo.environ import *

model = ConcreteModel()

# Decision Variables
model.N_servers = Var(domain=NonNegativeIntegers)
model.N_network_switches_poe = Var(domain=NonNegativeIntegers)
model.N_network_switches_non_poe = Var(domain=NonNegativeIntegers)
model.N_hard_drives = Var(domain=NonNegativeIntegers)
model.N_ssds = Var(domain=NonNegativeIntegers)

# Coefficients
beta1 = 0.05279680365
beta2_min, beta2_max = 25 / 1000 / 4, 62.5 / 1000 / 4  # convert Wh to kWh and per 15 min
beta3_min, beta3_max = 5 / 1000 / 4, 12.5 / 1000 / 4  # convert Wh to kWh and per 15 min
beta4_min, beta4_max = 0.175 / 1000 / 4, 0.2625 / 1000 / 4  # convert Wh to kWh and per 15 min
beta5 = 0.175 / 1000 / 4  # convert Wh to kWh and per 15 min

# Define ranges for coefficients to use the average for the constraints
beta2_avg = (beta2_min + beta2_max) / 2
beta3_avg = (beta3_min + beta3_max) / 2
beta4_avg = (beta4_min + beta4_max) / 2

# Total energy consumption constraint
model.total_energy_consumption = Constraint(expr=
    beta1 * model.N_servers +
    beta2_avg * model.N_network_switches_poe +
    beta3_avg * model.N_network_switches_non_poe +
    beta4_avg * model.N_hard_drives +
    beta5 * model.N_ssds
)

ERROR: Rule failed when generating expression for Constraint
total_energy_consumption with index None: ValueError: Constraint
'total_energy_consumption' does not have a proper value. Found
'0.05279680365*N_servers + 0.0109375*N_network_switches_poe +
0.0021875*N_network_switches_non_poe + 5.468750000000001e-05*N_hard_drives +
4.375e-05*N_ssds' Expecting a tuple or relational expression. Examples:
       sum(model.costs) == model.income (0, model.price[item], 50)
ERROR: Constructing component 'total_energy_consumption' from data=None
failed:
        ValueError: Constraint 'total_energy_consumption' does not have a
        proper value. Found '0.05279680365*N_servers +
        0.0109375*N_network_switches_poe +
        0.0021875*N_network_switches_non_poe +
        5.468750000000001e-05*N_hard_drives + 4.375e-05*N_ssds'
    Expecting a tuple or relational expression. Examples:
       sum(model.costs) == model.income (0, model.price[item], 50)


ValueError: Constraint 'total_energy_consumption' does not have a proper value. Found '0.05279680365*N_servers + 0.0109375*N_network_switches_poe + 0.0021875*N_network_switches_non_poe + 5.468750000000001e-05*N_hard_drives + 4.375e-05*N_ssds'
Expecting a tuple or relational expression. Examples:
   sum(model.costs) == model.income
   (0, model.price[item], 50)

To create a simple equation for the minimum number of active hardware based on the time of day, we can consider different times and the corresponding minimum requirements for each type of hardware. Let's define some time-based categories and their respective minimum requirements. For simplicity, let's divide the day into three periods: peak hours, off-peak hours, and regular hours.

### Time Periods:
1. **Peak Hours (e.g., 8 AM - 6 PM)**
2. **Regular Hours (e.g., 6 PM - 10 PM)**
3. **Off-Peak Hours (e.g., 10 PM - 8 AM)**

### Assumptions:
- Peak Hours require the highest number of active hardware.
- Regular Hours require a moderate number of active hardware.
- Off-Peak Hours require the lowest number of active hardware.

### Base Minimum Numbers:
These numbers should be adjusted based on your specific operational needs. For illustration:

- **Number of Servers:**
  - Peak Hours: 50
  - Regular Hours: 40
  - Off-Peak Hours: 30

- **Number of Network Switches (PoE):**
  - Peak Hours: 10
  - Regular Hours: 8
  - Off-Peak Hours: 6

- **Number of Network Switches (Non-PoE):**
  - Peak Hours: 5
  - Regular Hours: 4
  - Off-Peak Hours: 3

- **Number of Hard Drives (HDD):**
  - Peak Hours: 100
  - Regular Hours: 80
  - Off-Peak Hours: 60

- **Number of SSDs:**
  - Peak Hours: 50
  - Regular Hours: 40
  - Off-Peak Hours: 30

### Equations:
Let \( t \) be the time of day in hours (0-23).

1. **Number of Active Servers (\( N_s \))**:
\[ 
N_s = 
\begin{cases} 
50 & \text{if } 8 \leq t < 18 \\
40 & \text{if } 18 \leq t < 22 \\
30 & \text{if } 22 \leq t < 8 
\end{cases}
\]

2. **Number of Active Network Switches (PoE) (\( N_{ns\_poe} \))**:
\[ 
N_{ns\_poe} = 
\begin{cases} 
10 & \text{if } 8 \leq t < 18 \\
8 & \text{if } 18 \leq t < 22 \\
6 & \text{if } 22 \leq t < 8 
\end{cases}
\]

3. **Number of Active Network Switches (Non-PoE) (\( N_{ns\_non\_poe} \))**:
\[ 
N_{ns\_non\_poe} = 
\begin{cases} 
5 & \text{if } 8 \leq t < 18 \\
4 & \text{if } 18 \leq t < 22 \\
3 & \text{if } 22 \leq t < 8 
\end{cases}
\]

4. **Number of Active Hard Drives (HDD) (\( N_{hdd} \))**:
\[ 
N_{hdd} = 
\begin{cases} 
100 & \text{if } 8 \leq t < 18 \\
80 & \text{if } 18 \leq t < 22 \\
60 & \text{if } 22 \leq t < 8 
\end{cases}
\]

5. **Number of Active SSDs (\( N_{ssd} \))**:
\[ 
N_{ssd} = 
\begin{cases} 
50 & \text{if } 8 \leq t < 18 \\
40 & \text{if } 18 \leq t < 22 \\
30 & \text{if } 22 \leq t < 8 
\end{cases}
\]

Using these equations, you can adjust the minimum number of active hardware based on the time of day. You can tailor these numbers further based on your specific operational data and requirements.

In [ ]:
from pyomo.environ import ConcreteModel, Var, Constraint, NonNegativeIntegers, Objective, minimize

def get_min_active_hardware(time_of_day):
    if 8 <= time_of_day < 18:
        return {
            "servers": 50,
            "network_switches_poe": 10,
            "network_switches_non_poe": 5,
            "hard_drives": 100,
            "ssds": 50
        }
    elif 18 <= time_of_day < 22:
        return {
            "servers": 40,
            "network_switches_poe": 8,
            "network_switches_non_poe": 4,
            "hard_drives": 80,
            "ssds": 40
        }
    else:
        return {
            "servers": 30,
            "network_switches_poe": 6,
            "network_switches_non_poe": 3,
            "hard_drives": 60,
            "ssds": 30
        }

def create_model(time_of_day):
    min_hardware = get_min_active_hardware(time_of_day)
    
    model = ConcreteModel()
    
    # Define variables
    model.servers = Var(within=NonNegativeIntegers)
    model.network_switches_poe = Var(within=NonNegativeIntegers)
    model.network_switches_non_poe = Var(within=NonNegativeIntegers)
    model.hard_drives = Var(within=NonNegativeIntegers)
    model.ssds = Var(within=NonNegativeIntegers)
    
    # Define constraints based on minimum hardware requirements
    model.min_servers = Constraint(expr=model.servers >= min_hardware['servers'])
    model.min_network_switches_poe = Constraint(expr=model.network_switches_poe >= min_hardware['network_switches_poe'])
    model.min_network_switches_non_poe = Constraint(expr=model.network_switches_non_poe >= min_hardware['network_switches_non_poe'])
    model.min_hard_drives = Constraint(expr=model.hard_drives >= min_hardware['hard_drives'])
    model.min_ssds = Constraint(expr=model.ssds >= min_hardware['ssds'])
    
    # Example objective: Minimize total number of components
    model.objective = Objective(expr=model.servers + model.network_switches_poe + model.network_switches_non_poe + model.hard_drives + model.ssds, sense=minimize)
    
    return model

# Example usage:
time_of_day = 10  # Example time of day
model = create_model(time_of_day)

# Solve the model (you need a solver for this part)
from pyomo.opt import SolverFactory
solver = SolverFactory('glpk')  # Ensure you have the solver installed
result = solver.solve(model)

# Display results
model.servers.display()
model.network_switches_poe.display()
model.network_switches_non_poe.display()
model.hard_drives.display()
model.ssds.display()